In [78]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, to_date, date_format, to_timestamp, substring, max, avg
os.chdir('/Users/chkapsalis/Documents/GitHub/Big_Data_Architectures/Assignments/my_assignment_3')

# For some reason i need to run this every time in order to get it work
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home" 

# the .config("option", "value") arguments allow us to perform refined file I/O
spark = SparkSession.builder.master("local[1]") \
            .appName("app") \
            .config("option", "value") \
            .getOrCreate()

In [21]:
stocks = spark.read.csv('file:///' + os.getcwd() + '/all_stocks_5yr.csv', header=True)
stocks.printSchema()
stocks.show()

root
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- Name: string (nullable = true)

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26|13.14|13.42| 1

In [66]:
# "volume" should be considered a numeric column; changing
stocks = stocks.withColumn("volume", stocks["volume"].cast("int"))
stocks = stocks.withColumn("date", stocks["date"].cast("date"))

In [67]:
stocks.printSchema()
stocks.show()

root
 |-- date: date (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26|13.14|13.42| 12

In [ ]:
# Question 1

from pyspark.sql.functions import max, col

# Step 1: Compute max volume per stock
max_volume_per_stock = stocks.groupBy("Name") \
    .agg(max("volume").alias("max_volume"))

# Step 2: Alias both DataFrames before join
stocks_alias = stocks.alias("s")
max_vol_alias = max_volume_per_stock.alias("m")

# Step 3: Join and select clearly from the aliased versions
most_trans_per_ticker = stocks_alias \
    .join(
        max_vol_alias,
        (col("s.Name") == col("m.Name")) &
        (col("s.volume") == col("m.max_volume")),
        how="inner"
    ) \
    .select(
        col("s.Name").alias("Name"),
        col("s.date").alias("Date"),
        col("s.volume").alias("Volume")
    ) \
    .orderBy(col("Volume").desc())

most_trans_per_ticker.show()


+-----+----------+---------+
| Name|      Date|   Volume|
+-----+----------+---------+
|   VZ|2014-02-24|618237630|
|   GE|2015-11-17|431332632|
|  BAC|2016-02-11|375088650|
|   FB|2013-07-25|365380568|
|  PFE|2016-04-05|284468054|
|  MRO|2016-03-01|273996613|
|  AMD|2017-05-02|268336455|
| AAPL|2014-01-28|266833581|
|  KMI|2014-11-26|251563883|
| MSFT|2013-07-19|248354245|
| CSCO|2013-11-14|243255407|
|    F|2013-12-18|220362796|
| AMAT|2015-04-27|219415189|
|  DAL|2013-09-10|206363059|
|  CHK|2016-03-04|188966428|
| NFLX|2013-10-22|181099968|
|CMCSA|2014-02-13|166551204|
| ORCL|2013-07-12|157674694|
|   MU|2013-10-11|153906087|
|  FCX|2016-01-12|141597285|
+-----+----------+---------+
only showing top 20 rows



In [64]:
# OPTION B - SPARKSQL

stocks.createOrReplaceTempView("stocks")
query = spark.sql("""
    SELECT s.Name, s.date, s.volume
    FROM stocks s
    INNER JOIN (
        SELECT Name, MAX(volume) AS max_volume
        FROM stocks
        GROUP BY Name
    ) m
    ON s.Name = m.Name AND s.volume = m.max_volume
    ORDER BY s.volume DESC
""")

query.show()


+-----+----------+---------+
| Name|      date|   volume|
+-----+----------+---------+
|   VZ|2014-02-24|618237630|
|   GE|2015-11-17|431332632|
|  BAC|2016-02-11|375088650|
|   FB|2013-07-25|365380568|
|  PFE|2016-04-05|284468054|
|  MRO|2016-03-01|273996613|
|  AMD|2017-05-02|268336455|
| AAPL|2014-01-28|266833581|
|  KMI|2014-11-26|251563883|
| MSFT|2013-07-19|248354245|
| CSCO|2013-11-14|243255407|
|    F|2013-12-18|220362796|
| AMAT|2015-04-27|219415189|
|  DAL|2013-09-10|206363059|
|  CHK|2016-03-04|188966428|
| NFLX|2013-10-22|181099968|
|CMCSA|2014-02-13|166551204|
| ORCL|2013-07-12|157674694|
|   MU|2013-10-11|153906087|
|  FCX|2016-01-12|141597285|
+-----+----------+---------+
only showing top 20 rows



In [81]:
# Question 2
avg_volume_per_month = stocks \
    .select(
        date_format(col("date"), "MMM yyyy").alias("date"),
        col("volume")
    ) \
    .groupBy(col("date")) \
    .agg(avg("volume").alias("avg_volume")) \
    .orderBy(col("date"), ascending=True)

In [82]:
avg_volume_per_month.show()

+--------+------------------+
|    date|        avg_volume|
+--------+------------------+
|Apr 2013| 4990292.249235766|
|Apr 2014| 4712436.740587831|
|Apr 2015|4049128.5425065733|
|Apr 2016| 4403273.369344325|
|Apr 2017|3879889.4020380294|
|Aug 2013|3936255.4692568844|
|Aug 2014|  3277133.94670969|
|Aug 2015| 4910543.650503291|
|Aug 2016|3645052.5863453816|
|Aug 2017|3479362.4257066296|
|Dec 2013|4092123.8206644976|
|Dec 2014|3991512.5363263786|
|Dec 2015| 4406736.627640037|
|Dec 2016|4130377.3358465354|
|Dec 2017|3919747.9897999605|
|Feb 2013|  5261788.80732293|
|Feb 2014| 4918514.594747739|
|Feb 2015| 4254824.743407599|
|Feb 2016| 5756484.877016129|
|Feb 2017| 4255274.505789474|
+--------+------------------+
only showing top 20 rows



In [72]:
# OPTION B - SPARKSQL

r2 = spark.sql("""

    SELECT 
        date_format(date, "MMM yyyy") AS month,
        AVG(volume) AS avg_volume
    FROM stocks
    GROUP BY date_format(date, "MMM yyyy")
    ORDER BY month ASC

""")

In [73]:
r2.show()

+--------+------------------+
|   month|        avg_volume|
+--------+------------------+
|Apr 2013| 4990292.249235766|
|Apr 2014| 4712436.740587831|
|Apr 2015|4049128.5425065733|
|Apr 2016| 4403273.369344325|
|Apr 2017|3879889.4020380294|
|Aug 2013|3936255.4692568844|
|Aug 2014|  3277133.94670969|
|Aug 2015| 4910543.650503291|
|Aug 2016|3645052.5863453816|
|Aug 2017|3479362.4257066296|
|Dec 2013|4092123.8206644976|
|Dec 2014|3991512.5363263786|
|Dec 2015| 4406736.627640037|
|Dec 2016|4130377.3358465354|
|Dec 2017|3919747.9897999605|
|Feb 2013|  5261788.80732293|
|Feb 2014| 4918514.594747739|
|Feb 2015| 4254824.743407599|
|Feb 2016| 5756484.877016129|
|Feb 2017| 4255274.505789474|
+--------+------------------+
only showing top 20 rows



In [83]:
# Question 3
avg_volume_per_tick = stocks \
    .select(
        col("Name"),
        col("volume")
    ) \
    .groupBy(col("Name")) \
    .agg(avg("volume").alias("avg_volume")) \
    .orderBy(col("avg_volume"), ascending=False)

In [84]:
avg_volume_per_tick.show()

+-----+--------------------+
| Name|          avg_volume|
+-----+--------------------+
|  BAC| 9.363379951628277E7|
| AAPL| 5.404789973550437E7|
|   GE| 4.211568283240667E7|
|    F|  3.44623748943606E7|
|   FB| 3.435926520969023E7|
| MSFT| 3.386946274583002E7|
|  AMD| 3.251903779666402E7|
|   MU| 3.024841118903892E7|
| INTC|2.9326713900714852E7|
| CSCO|2.8654349631453533E7|
|  PFE|2.8218910923749007E7|
|    T|2.5298330478951547E7|
|  CHK|2.4957711172359016E7|
|CMCSA|2.4329297557585385E7|
|  FCX| 2.354921657426529E7|
|    C| 2.068675837807784E7|
|  WFC|1.8511480962668784E7|
|  JPM|1.6589033246227164E7|
|   RF|1.6398451548848292E7|
| NFLX|1.6208007963463066E7|
+-----+--------------------+
only showing top 20 rows



In [ ]:
# Question 4

# Step 0: Create a new DataFrame with the spread column
stocks_with_spread = stocks.withColumn("spread", col("high") - col("low"))

# Step 1: Compute max spread per stock
max_spread_per_stock = stocks_with_spread.groupBy("Name") \
    .agg(max("spread").alias("max_spread"))

# Step 2: Alias both DataFrames before join
stocks_alias = stocks_with_spread.alias("s")
max_spread_alias = max_spread_per_stock.alias("m")

# Step 3: Join and select desired columns
max_spread_dates = stocks_alias \
    .join(
        max_spread_alias,
        (col("s.Name") == col("m.Name")) &
        (col("s.spread") == col("m.max_spread")),
        how="inner"
    ) \
    .select(
        col("s.Name").alias("Name"),
        col("s.date").alias("Date"),
        col("s.spread").alias("Spread")
    ) \
    .orderBy(col("Spread").desc())

max_spread_dates.show()


+-----+----------+------------------+
| Name|      Date|            Spread|
+-----+----------+------------------+
| AMZN|2018-02-05|            138.26|
| PCLN|2017-11-07|109.90000000000009|
|  CMG|2015-11-20|              80.5|
|  AZO|2017-12-05|59.694999999999936|
| GOOG|2018-02-06| 58.57000000000005|
|GOOGL|2018-02-05|             58.25|
| REGN|2015-09-28| 56.90500000000003|
| PRGO|2015-04-08|51.097999999999985|
|  HUM|2015-05-29| 43.38499999999999|
| ULTA|2015-08-24|             43.22|
|  MCK|2015-08-24|           42.1799|
|  HCA|2015-08-24|             42.06|
| BIIB|2014-03-21|41.920000000000016|
| ALGN|2017-12-04|             35.25|
|  BLK|2018-02-05| 34.52999999999997|
|  ADS|2016-01-28| 33.56999999999999|
|  MTD|2018-02-05|           33.2595|
|   BA|2018-02-05|32.610000000000014|
|  AGN|2015-10-21|           32.4999|
|  TDG|2017-01-20|             32.25|
+-----+----------+------------------+
only showing top 20 rows



In [93]:
# Question 5
avg_volume_per_tick = stocks \
    .select(
        col("Name"),
        col("close")
    ) \
    .filter(date_format(col("date"), "yyyy") == 2016) \
    .groupBy(col("Name")) \
    .agg(avg("close").alias("avg_stock_price")) \
    .orderBy(col("avg_stock_price"), ascending=False) 

avg_volume_per_tick.show()

+-----+------------------+
| Name|   avg_stock_price|
+-----+------------------+
| PCLN|1346.7126984126985|
|  AZO| 768.8588492063492|
|GOOGL| 763.2142063492068|
| GOOG| 743.4851190476194|
| AMZN| 699.5231349206346|
|  CMG|428.56464285714287|
| REGN|394.49996031746036|
|  MTD| 375.5649206349207|
|  BLK|350.08837301587295|
| EQIX|343.77674603174603|
| BIIB| 280.1989682539682|
|  SHW| 277.9422222222223|
| ORLY|269.24904761904753|
|  TDG|251.80250000000007|
|  AGN|241.69857142857143|
|  AYI|241.64035714285708|
|  PSA|238.75015873015872|
|  LMT|237.99261904761906|
| CHTR|229.49678571428566|
| ULTA|224.08111111111108|
+-----+------------------+
only showing top 20 rows



In [95]:
# Question 6

all_tickers = stocks \
    .select(col("Name")) \
    .distinct() 

all_tickers.show()

+----+
|Name|
+----+
|ALXN|
| GIS|
|   K|
| LEN|
|SPGI|
| AIV|
| AVY|
|BF.B|
| MMM|
| PKI|
| PPG|
|  RF|
| AXP|
|  CI|
| IRM|
| WEC|
|INFO|
| PFG|
|  PM|
| SNA|
+----+
only showing top 20 rows

